In [1]:
from datetime import datetime, timedelta
from pathlib import Path

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.winds import BestTrackForcing
from adcircpy.utilities import download_mesh

import shapefile
import geopandas as gpd
from shapely.geometry import Point, Polygon

import numpy as np

stage = 'sims' #'staging/runs'
HURRICANE = 'Matthew2016'
OUTPUT_DIRECTORY = stage+'/Gonave_SLR_retreats/setup/r4_new/' #stage+'/runs/'+HURRICANE+'_v3c2_better_cover_rehab'
shapefile_path = 'datasets/mangrove_covers/refined/mangrove_polygon_refined.shp'

/home/aesgeorges/anaconda3/envs/adcircpy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def add_mannings_n_for_multiple_polygons(mesh, shapefile_n_pairs):
    """
    Add Manning's n values to mesh nodes within polygons from multiple shapefiles.
    
    Parameters:
    -----------
    mesh : adcircpy.AdcircMesh
        ADCIRC mesh object
    shapefile_n_pairs : list of tuples
        List of tuples where each tuple contains:
        - shapefile_path (str): Path to the shapefile containing polygons
        - mannings_n_value (float): Manning's n value to apply to all nodes within the polygons
    
    Returns:
    --------
    adcircpy.AdcircMesh
        The modified mesh object with updated Manning's n values
    """
    default_n = 0.025  # Default Manning's n value

    # Initialize Manning's n if it doesn't exist
    #if not hasattr(mesh, 'mannings_n_at_sea_floor'):
     #   mesh.mannings_n_at_sea_floor = np.full(len(mesh.coords), default_n)
    
    # Track total modified nodes
    total_modified = 0
    
    node_values = np.full(mesh.values.shape, 0.025)
    # Process each shapefile and its corresponding Manning's n value
    for shapefile_path, mannings_n_value in shapefile_n_pairs:
        print(f"Processing shapefile: {shapefile_path} with Manning's n = {mannings_n_value}")
        
        # Load polygons from shapefile
        polygons = gpd.read_file(shapefile_path)
        print(f"Loaded {len(polygons)} polygons")
        
        # Process each polygon
        for idx, row in polygons.iterrows():
            polygon = row.geometry
            
            # Find and update nodes within this polygon
            count = 0
            if polygon is not None:
                for i, node_row in mesh.coords.iterrows():
                    x = node_row['x']  # Assuming column is named 'x'
                    y = node_row['y']  # Assuming column is named 'y'
                    
                    point = Point(x, y)
                    if polygon.contains(point):
                        node_values[i-1] = mannings_n_value
                        count += 1
            else:
                print(f"Polygon {idx} is None, skipping...")
                continue
            
            total_modified += count
            print(f"Polygon {idx}: Updated {count} nodes with Manning's n = {mannings_n_value}")
    # Update the mesh's mannings_n_at_sea_floor array
    mesh.mannings_n_at_sea_floor = node_values
    
    print(f"Total nodes modified with non-default values: {total_modified}")
    return mesh

In [3]:
mangrove_n = 0.2 #0.2 for mangrove
mudflat_n = 0.016
sand_n = 0.015
urban_n = 0.15
crop_n = 0.08

shapefile_root = 'datasets/mangrove_covers/refined/'
# Define the shapefile and corresponding Manning's n values
pairs = [[shapefile_root+'urban_polygon_refined.shp', urban_n],
         [shapefile_root+'mangrove_polygon_retreat_medium2.shp', mangrove_n],
         [shapefile_root+'mudflat_polygon_refined.shp', mudflat_n],
         [shapefile_root+'sand_polygon_refined.shp', sand_n],
         [shapefile_root+'crop_polygon_refined.shp', crop_n]]

In [4]:
# open mesh file
mesh = AdcircMesh.open('sims/Gonave_SLR_retreats/r0/S0/fort.14', crs=4326)    # OR 'datasets/meshes/ww_gonave_v3c.grd'

# Adding Manning's
# generate tau0 factor
mesh.generate_tau0()

# also add Manning's N to the domain (constant for this example)
#mesh.mannings_n_at_sea_floor = np.full(mesh.values.shape, 0.025)
mesh = add_mannings_n_for_multiple_polygons(mesh, pairs)
# initialize tidal forcing and constituents
#tidal_forcing = Tides()
#tidal_forcing.use_all()
#mesh.add_forcing(tidal_forcing)

/home/aesgeorges/anaconda3/envs/adcircpy/lib/python3.11/site-packages/adcircpy/mesh/parsers/grd.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['nodes'] = pandas.read_csv(
/home/aesgeorges/anaconda3/envs/adcircpy/lib/python3.11/site-packages/adcircpy/mesh/parsers/grd.py:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['elements'] = pandas.read_csv(


Processing shapefile: datasets/mangrove_covers/refined/urban_polygon_refined.shp with Manning's n = 0.15
Loaded 7 polygons
Polygon 0: Updated 150 nodes with Manning's n = 0.15
Polygon 1: Updated 5 nodes with Manning's n = 0.15
Polygon 2: Updated 0 nodes with Manning's n = 0.15
Polygon 3: Updated 0 nodes with Manning's n = 0.15
Polygon 4: Updated 0 nodes with Manning's n = 0.15
Polygon 5: Updated 0 nodes with Manning's n = 0.15
Polygon 6: Updated 0 nodes with Manning's n = 0.15
Processing shapefile: datasets/mangrove_covers/refined/mangrove_polygon_retreat_medium2.shp with Manning's n = 0.2
Loaded 1 polygons
Polygon 0: Updated 689 nodes with Manning's n = 0.2
Processing shapefile: datasets/mangrove_covers/refined/mudflat_polygon_refined.shp with Manning's n = 0.016
Loaded 4 polygons
Polygon 0: Updated 470 nodes with Manning's n = 0.016
Polygon 1 is None, skipping...
Polygon 2 is None, skipping...
Polygon 3 is None, skipping...
Processing shapefile: datasets/mangrove_covers/refined/sand_

In [5]:
# set simulation dates
spinup_time = timedelta(days=15)                                                                        
duration = timedelta(days=11)
start_date = datetime(2016, 9, 28, 12)
end_date = start_date + duration

In [6]:
# initialize wind forcing
wind_forcing = BestTrackForcing(storm=HURRICANE, nws=20)
mesh.add_forcing(wind_forcing)

/home/aesgeorges/anaconda3/envs/adcircpy/lib/python3.11/site-packages/stormevents/nhc/track.py:1137: RuntimeWarning: invalid value encountered in divide
  speeds = pandas.Series(distances / abs(intervals), index=indices)


In [7]:
# instantiate driver object
driver = AdcircRun(mesh, start_date, end_date, spinup_time, netcdf=True)   
driver.NOUTGE = -3
driver.NOUTGV = -3
driver.NOUTGM = -3
driver.TOUTFGE = 30
driver.TOUTFGV = 30
driver.TOUTFGM = 30
driver.NSPOOLGE = 1800
driver.NSPOOLGV = 1800
driver.NSPOOLGM = 1800
driver.DTDP = 10
# write driver state to disk
driver.write(OUTPUT_DIRECTORY, overwrite=True)

In [8]:
bash_script = rf"""
#!/bin/bash

current_dir=$(pwd)
subdomain_dir=$current_dir/s1_gonave
folder_name=$(basename $current_dir)

mkdir ../../logs
mkdir ../../logs/$folder_name
log_header="Logs of run attempts and their parameters."
log_header2="Hurricane $folder_name"
log_header3="------------------------------------------"
log_header4="id\t\t\t\t\t\tdt\t\t\t\t\t\ttype\t\t\t\t\t\tcomment"
echo -e $log_header > runlog.out
echo -e $log_header2 >> runlog.out
echo -e $log_header3 >> runlog.out
echo -e $log_header4 >> runlog.out

cp ../../../scripts/fort22fix.py fort22fix.py
python fort22fix.py

cp ../../../scripts/run_new.sh run.sh
cp ../../../scripts/slurmcold.sh slurm.sh

current_dir=$(pwd)
folder_name=$(basename $current_dir)
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name/" slurm.sh

# subdomain file structure setup
mkdir s1_gonave
subdomain_dir=$current_dir/s1_gonave
cp ../../../datasets/s1_gonave/shape.e14 s1_gonave/shape.e14
# remove BOM (\ufeff) from shape.e14
sed -i '1s/^\xef\xbb\xbf//' s1_gonave/shape.e14

source ~/anaconda3/etc/profile.d/conda.sh  # Adjust the path to your Conda installation
conda activate py2
python2.7 ../../../scripts/gensub.py $current_dir $subdomain_dir

chmod +x run.sh slurm.sh 

"""

# Write the bash script to the output directory
with open(OUTPUT_DIRECTORY + '/init.sh', 'w') as file:
    file.write(bash_script)

print("Bash script created and outputted at", OUTPUT_DIRECTORY)

Bash script created and outputted at sims/Gonave_SLR_retreats/setup/r4_new/
